# 🐙 Using Rubrix and Snorkel for human-in-the-loop weak supervision

In this tutorial, we will walk through the process of using Rubrix to improve weak supervision and data programming workflows with the amazing Snorkel library.

## Introduction

**Our goal is to show you how you can incorporate Rubrix into data programming workflows** to programatically build training data with a human-in-the-loop approach. We will use the widely-known [Snorkel](https://www.snorkel.org/) library, but a similar approach can be used with other data augmentation libraries such as [Textattack](https://github.com/QData/TextAttack) or [nlpaug](https://github.com/makcedward/nlpaug).

### What is weak supervision? and Snorkel?

Weak supervision is a branch of machine learning based on getting lower quality labels more efficiently. We can achieve this by using Snorkel, a library for programmatically building and managing training datasets without manual labeling.

### This tutorial

In this tutorial, we will follow the [Spam classification tutorial](https://www.snorkel.org/use-cases/01-spam-tutorial) from Snorkel's documentation and show you how to extend weak supervision workflows with Rubrix. 

The tutorial is organized into:

1. **Spam classification with Snorkel**: we provide a brief overview of the tutorial

2. **Extending and finding labeling functions with Rubrix**: we analyze different strategies for extending the proposed labeling functions and for exploring new labeling functions

## Install Snorkel, Textblob and spaCy

In [3]:
!pip install snorkel textblob spacy -qqq

In [5]:
!python -m spacy download en_core_web_sm -qqq

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# 2. Spam classification with Snorkel

Rubrix allows you to log and track data for different NLP tasks (such as `Token Classification` or `Text Classification`). 

In this tutorial, we will use the [YouTube Spam Collection](http://www.dt.fee.unicamp.br/~tiago//youtubespamcollection/) dataset which a binary classification task for detecting spam comments in youtube videos.

## The dataset

We have 2 datasets, the training and the test. The first one does not include the label of the samples and it is set to -1. Something different happens with the test, where the label is set to 1 if it's considered SPAM and 0 for HAM.  

Let's load it in Pandas and take a look!

In [1]:
import pandas as pd
df_train = pd.read_csv('yt_comments_train.csv')
df_test = pd.read_csv('yt_comments_test.csv')
display(df_train)
display(df_test)

,Unnamed: 0,author,date,text,label,video
0,0,Alessandro leite,2014-11-05T22:21:36,pls http://www10.vakinha.com.br/VaquinhaE.aspx...,-1.0,1
1,1,Salim Tayara,2014-11-02T14:33:30,"if your like drones, plz subscribe to Kamal Ta...",-1.0,1
2,2,Phuc Ly,2014-01-20T15:27:47,go here to check the views :3﻿,-1.0,1
3,3,DropShotSk8r,2014-01-19T04:27:18,"Came here to check the views, goodbye.﻿",-1.0,1
4,4,css403,2014-11-07T14:25:48,"i am 2,126,492,636 viewer :D﻿",-1.0,1
...,...,...,...,...,...,...
1581,443,Themayerlife,NaN,Check out my mummy chanel!,-1.0,4
1582,444,Fill Reseni,2015-05-27T17:10:53.724000,The rap: cool Rihanna: STTUUPID﻿,-1.0,4
1583,445,Greg Fils Aimé,NaN,I hope everyone is in good spirits I&#39;m a h...,-1.0,4
1584,446,Lil M,NaN,Lil m !!!!! Check hi out!!!!! Does live the wa...,-1.0,4


,Unnamed: 0,author,date,text,label,video
0,27,‫حلم الشباب‬‎,2015-05-25T23:42:49.533000,Check out this video on YouTube:﻿,1,5
1,194,MOHAMED THASLEEM,2015-05-24T07:03:59.488000,super music﻿,0,5
2,277,AlabaGames,2015-05-22T00:31:43.922000,Subscribe my channel I RECORDING FIFA 15 GOAL...,1,5
3,132,Manish Ray,2015-05-23T08:55:07.512000,This song is so beauty,0,5
4,163,Sudheer Yadav,2015-05-28T10:28:25.133000,SEE SOME MORE SONG OPEN GOOGLE AND TYPE Shakir...,1,5
...,...,...,...,...,...,...
245,32,GamezZ MTA,2015-05-09T00:08:26.185000,Pleas subscribe my channel﻿,1,5
246,176,Viv Varghese,2015-05-25T08:59:50.837000,The best FIFA world cup song for sure.﻿,0,5
247,314,yakikukamo FIRELOVER,2013-07-18T17:07:06.152000,hey you ! check out the channel of Alvar Lake !!,1,5
248,25,James Cook,2013-10-10T18:08:07.815000,Hello Guys...I Found a Way to Make Money Onlin...,1,5


## Labeling functions

Labeling functions (LFs) are heuristics that take a data point as input and assign a label to it or abstain (don’t assign any label).
LFs are noisy (they might not have perfect accuracy) and don't have to label every data point.

Some of the most common LFs rely on:

- Keyword searches: looking for specific words in a sentence
- Pattern matching: looking for specific syntactical patterns
- Heuristics: slight improvements based on certain assumptions
- Third-party models: using an pre-trained model (usually a model for a different task than the one at hand)
- Distant supervision: using external knowledge base
- Crowdworker labels: treating each crowdworker as a black-box function that assigns labels to subsets of the data

More information can be found in the [Snorkel LFs tutorial](https://www.snorkel.org/use-cases/01-spam-tutorial#a-gentle-introduction-to-lfs)

Let's see how we can use some of this methods to make LFs:

In [2]:
import re

from snorkel.labeling import labeling_function, LabelingFunction
from snorkel.labeling.lf.nlp import nlp_labeling_function
from snorkel.preprocess import preprocessor
from snorkel.preprocess.nlp import SpacyPreprocessor

from textblob import TextBlob


ABSTAIN = -1
HAM = 0
SPAM = 1

# Keyword searches
@labeling_function()
def check(x):
    return SPAM if "check" in x.text.lower() else ABSTAIN

@labeling_function()
def check_out(x):
    return SPAM if "check out" in x.text.lower() else ABSTAIN

# Heuristics
@labeling_function()
def short_comment(x):
    """Ham comments are often short, such as 'cool video!'"""
    return HAM if len(x.text.split()) < 5 else ABSTAIN

# List of keywords
def keyword_lookup(x, keywords, label):
    if any(word in x.text.lower() for word in keywords):
        return label
    return ABSTAIN

def make_keyword_lf(keywords, label=SPAM):
    return LabelingFunction(
        name=f"keyword_{keywords[0]}",
        f=keyword_lookup,
        resources=dict(keywords=keywords, label=label),
    )

"""Spam comments talk about 'my channel', 'my video', etc."""
keyword_my = make_keyword_lf(keywords=["my"])

"""Spam comments ask users to subscribe to their channels."""
keyword_subscribe = make_keyword_lf(keywords=["subscribe"])

"""Spam comments post links to other channels."""
keyword_link = make_keyword_lf(keywords=["http"])

"""Spam comments make requests rather than commenting."""
keyword_please = make_keyword_lf(keywords=["please", "plz"])

"""Ham comments actually talk about the video's content."""
keyword_song = make_keyword_lf(keywords=["song"], label=HAM)


# Pattern matching with regex
@labeling_function()
def regex_check_out(x):
    return SPAM if re.search(r"check.*out", x.text, flags=re.I) else ABSTAIN


# Third party models (TextBlob and spaCy)
# TextBlob
@preprocessor(memoize=True)
def textblob_sentiment(x):
    scores = TextBlob(x.text)
    x.polarity = scores.sentiment.polarity
    x.subjectivity = scores.sentiment.subjectivity
    return x

@labeling_function(pre=[textblob_sentiment])
def textblob_subjectivity(x):
    return HAM if x.subjectivity >= 0.5 else ABSTAIN

@labeling_function(pre=[textblob_sentiment])
def textblob_polarity(x):
    return HAM if x.polarity >= 0.9 else ABSTAIN

# spaCy

# There are two different methods to use spaCy:
# Method 1:
spacy = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)

@labeling_function(pre=[spacy])
def has_person(x):
    """Ham comments mention specific people and are short."""
    if len(x.doc) < 20 and any([ent.label_ == "PERSON" for ent in x.doc.ents]):
        return HAM
    else:
        return ABSTAIN
    
# Method 2:
@nlp_labeling_function()
def has_person_nlp(x):
    """Ham comments mention specific people."""
    if any([ent.label_ == "PERSON" for ent in x.doc.ents]):
        return HAM
    else:
        return ABSTAIN

In [3]:
# List of labeling functions
lfs = [
    keyword_my,
    keyword_subscribe,
    keyword_link,
    keyword_please,
    keyword_song,
    regex_check_out,
    short_comment,
    has_person_nlp,
    textblob_polarity,
    textblob_subjectivity,
]

We have mentioned multiple functions that could be used to label our data, but we never gave a solution on how to deal with the overlap and conflicts. In this section, we will deal with this problem.  
A simple baseline for doing this is to take the majority vote on a per-data point basis: if more LFs voted SPAM than HAM, label it SPAM (and vice versa). We can test this with the [MajorityLabelVoter](https://snorkel.readthedocs.io/en/master/packages/_autosummary/labeling/snorkel.labeling.model.baselines.MajorityLabelVoter.html#snorkel.labeling.model.baselines.MajorityLabelVoter) baseline model implemented in Snorkel.  
The major inconvenience of this approach is that we defined rules that could be correlated, resulting in certain signals being overrepresented in a majority-vote-based model.  
To handle this issue, we are going to make use of the LabelModel. You can read more about how it works in the [Snorkel tutorial](https://www.snorkel.org/use-cases/01-spam-tutorial#4-combining-labeling-function-outputs-with-the-label-model) and the [documentation](https://snorkel.readthedocs.io/en/master/packages/_autosummary/labeling/snorkel.labeling.model.label_model.LabelModel.html#snorkel.labeling.model.label_model.LabelModel).

In [4]:
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel

# Apply LFs to datasets
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_test = applier.apply(df=df_test)

from snorkel.labeling.model import MajorityLabelVoter

majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train) # y_train labels
Y_test = df_test.label.values # y_test labels

label_model = LabelModel(cardinality=2, verbose=True) # cardinality = nº of classes
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)

majority_acc = majority_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

/home/javispp/miniconda3/envs/biome2/lib/python3.7/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 250/250 [00:02<00:00, 101.47it/s]
/home/javispp/miniconda3/envs/biome2/lib/python3.7/site-packages/torch/autograd/__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag


Majority Vote Accuracy:   81.2%
Label Model Accuracy:     86.4%


# 2. Extending and finding labeling functions with Rubrix

## Setup Rubrix

[here we should point the user to the install and setup guide]

And then show how to start with local or remote install.

By default, rubrix will make a local initialization (as shown in the setup guide). If you want to specify an API url and key, you can pass that information via two environment variables: `RUBRIX_API_KEY` and `RUBRIX_API_URL`.

In [ ]:
import rubrix as rb

rb.init()

### Exploring the training set with Rubrix for initial inspiration

Rubrix lets you log and track data for different NLP tasks (such as *Token Classification* or *Text Classification*). 

First of all, we have to create the dataset and load it into rubrix

In [6]:
records= []

for index, record in df_train.iterrows():     
    item = rb.TextClassificationRecord(
        id=index,
        inputs={"text": record["text"]},
        metadata = {
            "textlen": str(len(record.text)), 
            "author": record.author,
            "video": str(record.video)
        }
    )
    records.append(item)

In [7]:
rb.log(records=records, name="yt_spam_snorkel")

BulkResponse(dataset='yt_spam_snorkel', processed=1586, failed=0)

Once we have it into rubrix, we can explore the data and the metadata we just added by pressing the "view metadata" button on the bottom right corner of each sample. Take a look at the following picture:

<img src="metadata_view.png">

Now, we can explore the metadata by going to the top left. Let's say we want to check the different authors or inspect all the data with a specific one. By going to the author section, we can easily do all of this.

<img src="metadata_author.png">

After applying the changes we should only see the comments that belong to the selected author. We can also add multiple conditions besides the name of the author, but we will keep it simple this time.  
Another option is to use the top right search box to find samples that contain a certain word or a phrase ("put inside parantheses"). We will search for "check", since it is more likely that those comments are SPAM. 

<img src="search_check.png">

As we can see, the LF used before to label the samples with the word check as SPAM makes sense.

An LF development cycle with Rubrix, could look like this:
    
1. Load dataset into Rubrix
2. Explore data and write an initial version of an LF
3. Spot check its performance by looking at its output on data points in the training set (or development set if available)
4. Refine and debug to improve coverage or accuracy as necessary

### Exploring and improving heuristic LFs

We have already seen how to use keywords to label our data, the next step would be to use heuristics to do the labeling. A simple approach to this could be setting a minimum length to the comment, considering it SPAM if its length is lower than a threshold.  
To use the right threshold we are going to explore our data in Rubrix by using the metadata field, similar to what we did before with the author selection.

In [26]:
records= []

for index, record in df_train.iterrows():     
    item = rb.TextClassificationRecord(
        id=index,
        inputs={"text": record["text"]},
        metadata = {
            "textlen": str(len(record.text.split())), # Nº of 'words' in the sample
        }
    )
    records.append(item)

In [27]:
rb.log(records=records, name="yt_spam_snorkel_heuristic")

BulkResponse(dataset='yt_spam_snorkel_heuristic', processed=1586, failed=0)

For this example we will use a threshold of 20 'words'.

In [4]:
# this is for now the only LF we change
@labeling_function()
def short_comment_2(x):
    """Ham comments are often short, such as 'cool video!'"""
    return HAM if len(x.text.split()) < 20 else ABSTAIN

### Writing Keyword LFs with Rubrix

Now, we will use Rubrix to find improved LFs using lists of keywords.

In [32]:
records= []

for index, record in df_train.iterrows():     
    item = rb.TextClassificationRecord(
        id=index,
        inputs={"text": record["text"]},
    )
    records.append(item)

In [33]:
rb.log(records=records, name="yt_spam_snorkel_lfs")

BulkResponse(dataset='yt_spam_snorkel_lfs', processed=1586, failed=0)

In [5]:
"""Ham comments usually ask how something happened/is because they are interested."""
keyword_how = make_keyword_lf(keywords=["how"], label=HAM)

### Exploring third-party models LFs with Rubrix


#### Textblob

Let's explore Textblob predictions on the training set with Rubrix:

In [27]:
# TODO: I would check this part to work with confidence intervals once we can set the multilabel attribute
# to True. Otherwise, we have to load the same dataset for each label
from textblob import TextBlob

records= []
for index, record in df_train.iterrows():   
    scores = TextBlob(record["text"])
    item = rb.TextClassificationRecord(
        id=str(index),
        inputs={"text": record["text"]},
        multi_label= False,
        prediction=[("subjectivity", max(0.0, scores.sentiment.subjectivity))],
        #prediction=[("polarity", max(0.0, scores.sentiment.polarity))],
        prediction_agent="TextBlob",
        metadata = {
            "textlen": str(len(record.text)), 
            "author": record.author,
            "video": str(record.video)
        }
    )
    
    records.append(item)

In [28]:
rb.log(records=records, name="yt_spam_snorkel_textblob")

BulkResponse(dataset='yt_spam_snorkel_textblob', processed=1586, failed=0)

Checking the dataset, we can filter our data based on the confidence of our classifier. This can help us since the predictions of our TextBlob tend to be SPAM the lower the subjectivity is. We can take advantage of this by filtering the predictions using intervals of confidence. For this example we are going to set a threshold of 0.56 for the subjectivity.  

<img src="confidence_interval.png">

The same way we did it for subjectivity, we can do it for polarity. In this case we will set the threshold to 0.9.
This comes in handy when we want to add restrictions on top of another. This time we won't go much deeper, but we could also use metadata or create more than what we have, in combination with the confidence, to have a better understanding of the data.

Once we have our conclusions, we can proceed with creating the labeling functions using the thresholds for the confidence.

In [6]:
from snorkel.preprocess import preprocessor
from textblob import TextBlob


@preprocessor(memoize=True)
def textblob_sentiment(x):
    scores = TextBlob(x.text)
    x.polarity = scores.sentiment.polarity
    x.subjectivity = scores.sentiment.subjectivity
    return x

@labeling_function(pre=[textblob_sentiment])
def textblob_subjectivity(x):
    return HAM if x.subjectivity >= 0.56 else ABSTAIN

@labeling_function(pre=[textblob_sentiment])
def textblob_polarity(x):
    return HAM if x.polarity >= 0.9 else ABSTAIN

#### spaCy

For this example, we will use spaCy for a token classification problem.

In [8]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [ ]:
records = []
for index, record in df_train[40:].iterrows():
    doc = nlp(record["text"])
    item = rb.TokenClassificationRecord(
        text=record["text"],
        tokens=[t.text for t in doc],
        prediction=[(e.label_, e.start_char, e.end_char) for e in doc.ents],
        prediction_agent="spacy"
    )
    print(item)
    rb.log(item, name="yt_spam_snorkel_spacy")

In [ ]:
rb.log(records, name="yt_spam_snorkel_spacy")

It's time to explore Rubrix and see if there are some entities that are more common in the SPAM or HAM comments.

<img src="spacy_ner.png">

Exploring a bit the data, we will create a rule to predict as HAM if the sample has a PERSON entity in its text.

In [7]:
@nlp_labeling_function()
def has_person_nlp(x):
    """Ham comments mention specific people and are short."""
    if any([ent.label_ == "PERSON" for ent in x.doc.ents]):
        return HAM
    else:
        return ABSTAIN

## 3. Using the Snorkel Label Model

Let's include the new LFs we created using Rubrix

In [38]:
lfs = [
    keyword_my,
    keyword_subscribe,
    keyword_link,
    keyword_please,
    keyword_song,
    keyword_how,
    regex_check_out,
    short_comment,
    short_comment_2,
    has_person_nlp,
    textblob_polarity,
    textblob_subjectivity,
]

And apply it to our training and test datasets

In [39]:
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_test = applier.apply(df=df_test)

/home/javispp/miniconda3/envs/biome2/lib/python3.7/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 250/250 [00:00<00:00, 6840.38it/s]


In [40]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
keyword_my,0,[1],0.198613,0.197352,0.174653
keyword_subscribe,1,[1],0.127364,0.122951,0.106557
keyword_http,2,[1],0.119168,0.116646,0.110971
keyword_please,3,[1],0.112232,0.111602,0.095208
keyword_song,4,[0],0.141866,0.135561,0.043506
keyword_how,5,[0],0.036570,0.033417,0.012610
regex_check_out,6,[1],0.233922,0.229508,0.213745
short_comment,7,[0],0.225725,0.225725,0.074401
short_comment_2,8,[0],0.810845,0.688525,0.372636
has_person_nlp,9,[0],0.142497,0.138083,0.075662


In [41]:
from snorkel.labeling.model import MajorityLabelVoter

majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train) # y_train labels
Y_test = df_test.label.values # y_test labels

In [42]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True) # cardinality = nº of classes
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)

In [43]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True) # cardinality = nº of classes
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)

majority_acc = majority_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

Majority Vote Accuracy:   75.6%
Label Model Accuracy:     89.6%


As we can see, the LabelModel outperforms the basic MajorityLabelVoter configuration.

#### Filtering unlabeled data

The method we saw above, has a small inconvenience, our data can receive no prediction (predicted as ABSTAIN) from any of our LFs. Those cases with no label (labeled as ABSTAIN) will be removed from the training dataset using a [built-in utility](https://snorkel.readthedocs.io/en/master/packages/_autosummary/labeling/snorkel.labeling.filter_unlabeled_dataframe.html#snorkel.labeling.filter_unlabeled_dataframe).

In [44]:
from snorkel.labeling import filter_unlabeled_dataframe

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=df_train,
    y=label_model.predict_proba(L_train), # Probabilities of each data point for each class
    L=L_train
)

Now that we have our data, we can explore the results in Rubrix and manually relabel those cases that have been wrongly classified or keep exploring the performance of our LFs.

In [ ]:
records = []
i=0
for index, record in df_train_filtered.iterrows():   
    item = rb.TextClassificationRecord(
        id=str(index),
        inputs={"text": record["text"]},
        multilabel=True,
        # our confidences/scores come from probs_train_filtered
        # probs_train_filtered[i][j] is the probability the sample i belongs to class j
        prediction=[("HAM", probs_train_filtered[i][0]),   # 0 for HAM
                    ("SPAM", probs_train_filtered[i][1])], # 1 for SPAM
        prediction_agent="LabelModel",
        metadata = {
            "textlen": str(len(record.text)), 
            "author": record.author,
            "video": str(record.video)
        }
    )
    records.append(item)
    i+=1

In [ ]:
rb.log(records=records, name="yt_filtered_classified_sample")

To relabel the data we switch into the annotation mode by pressing the bottom right button in the rubrix UI. 
After that, we can simply click on the label for each sample and it will be marked as a "validated sample".

## 4. Train a classifier and explore its predictions

The last thing we can do with our data is training a classifier using some of the most popular libraries such as Scikit-learn, Tensorflow or Pytorch. For simplicity, we will use Scikit-learn, a well known library in ML.  
We recommend checking [biome.txt](https://www.recogn.ai/biome-text/), a simple and powerful tool for NLP problems, developed by the same team behind Rubrix.

In [45]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1, 5)) # Bag Of Words (BoW) with n-grams
X_train = vectorizer.fit_transform(df_train_filtered.text.tolist())
X_test = vectorizer.transform(df_test.text.tolist())

Since we need to tell the model the class for each sample, and we have probabilities, we can assign to each sample the class with the highest probability.

In [46]:
from snorkel.utils import probs_to_preds

preds_train_filtered = probs_to_preds(probs=probs_train_filtered)

And then build the classifier

In [47]:
from sklearn.linear_model import LogisticRegression

sklearn_model = LogisticRegression(C=1e3, solver="liblinear")
sklearn_model.fit(X=X_train, y=preds_train_filtered)

LogisticRegression(C=1000.0, solver='liblinear')

In [48]:
print(f"Test Accuracy: {sklearn_model.score(X=X_test, y=Y_test) * 100:.1f}%")

Test Accuracy: 91.2%


Let's see how our new model performs:

In [ ]:
records = []
for index, record in df_train_filtered.iterrows(): 
    preds=sklearn_model.predict_proba(vectorizer.transform([record["text"]]))
    item = rb.TextClassificationRecord(
        id=str(index),
        inputs={"text": record["text"]},
        multilabel=True,
        prediction=[("HAM", preds[0]),   # 0 for HAM
                    ("SPAM", preds[1])], # 1 for SPAM
        prediction_agent="MyModel",
    )
    records.append(item)

In [ ]:
rb.log(records=records, name="yt_my_model_performance")

Last but not least, we will show a simple model for text classification using biome.text.

In [ ]:
!pip install -U biome-text

But first, let's use the original name for the labels instead of numbers

In [49]:
# Change the label column to the class the sample belongs (HAM or SPAM)
df_test["label"] = df_test["label"].apply(lambda x: "SPAM" if x==1 else "HAM")

In [ ]:
# Change the label column to the class the sample belongs (HAM or SPAM) according to our LFs
df_train_filtered["label"] = list(map(lambda x: "SPAM" if x==1 else "HAM", preds_train_filtered))

In [51]:
from biome.text import Pipeline, Dataset

ds_train = Dataset.from_pandas(df_train_filtered)
ds_test = Dataset.from_pandas(df_test)

pipeline = Pipeline.from_config({
    "name": "my-first-classifier",
    "head": {"type": "TextClassification", "labels": list(set(df_test["label"]))}
})

training_results = pipeline.train(
    output="path_to_store_training_run_output",
    training=ds_train,
    test=ds_test
)

INFO:allennlp.data.vocabulary: Fitting token dictionary from dataset.
2021-04-28 12:21:56,585 - allennlp.data.vocabulary - INFO - Fitting token dictionary from dataset.



INFO:allennlp.modules.token_embedders.embedding: If you are fine-tuning and want to use a pretrained_file for embedding extension, please pass the mapping by --embedding-sources argument.
2021-04-28 12:21:56,658 - allennlp.modules.token_embedders.embedding - INFO - If you are fine-tuning and want to use a pretrained_file for embedding extension, please pass the mapping by --embedding-sources argument.
INFO:allennlp.common.params: random_seed = 13370
2021-04-28 12:21:56,668 - allennlp.common.params - INFO - random_seed = 13370
INFO:allennlp.common.params: numpy_seed = 1337
2021-04-28 12:21:56,673 - allennlp.common.params - INFO - numpy_seed = 1337
INFO:allennlp.common.params: pytorch_seed = 133
2021-04-28 12:21:56,675 - allennlp.common.params - INFO - pytorch_seed = 133
INFO:allennlp.common.checks: Pytorch version: 1.7.1
2021-04-28 12:21:56,682 - allennlp.common.checks - INFO - Pytorch version: 1.7.1
INFO:allennlp.common.params: type = gradient_descent
2021-04-28 12:21:56,712 - allennl

INFO:allennlp.training.trainer: Beginning training.
2021-04-28 12:21:59,426 - allennlp.training.trainer - INFO - Beginning training.
INFO:allennlp.training.trainer: Epoch 0/19
2021-04-28 12:21:59,431 - allennlp.training.trainer - INFO - Epoch 0/19
INFO:allennlp.training.trainer: Worker 0 memory usage: 1.1G
2021-04-28 12:21:59,437 - allennlp.training.trainer - INFO - Worker 0 memory usage: 1.1G
INFO:allennlp.training.trainer: Training
2021-04-28 12:21:59,457 - allennlp.training.trainer - INFO - Training


2021-04-28 12:21:59,539 - allennlp.training.util - WARNING - Metrics with names beginning with "_" will not be logged to the tqdm progress bar.

INFO:allennlp.training.tensorboard_writer:                        Training |  Validation
2021-04-28 12:22:00,633 - allennlp.training.tensorboard_writer - INFO -                        Training |  Validation
INFO:allennlp.training.tensorboard_writer: _fscore/HAM        |     0.711  |       N/A
2021-04-28 12:22:00,636 - allennlp.training.tensorboard_writer - INFO - _fscore/HAM        |     0.711  |       N/A
INFO:allennlp.training.tensorboard_writer: _fscore/SPAM       |     0.796  |       N/A
2021-04-28 12:22:00,639 - allennlp.training.tensorboard_writer - INFO - _fscore/SPAM       |     0.796  |       N/A
INFO:allennlp.training.tensorboard_writer: _precision/HAM     |     0.817  |       N/A
2021-04-28 12:22:00,646 - allennlp.training.tensorboard_writer - INFO - _precision/HAM     |     0.817  |       N/A
INFO:allennlp.training.tensorboard_writ


INFO:allennlp.training.tensorboard_writer:                        Training |  Validation
2021-04-28 12:22:01,663 - allennlp.training.tensorboard_writer - INFO -                        Training |  Validation
INFO:allennlp.training.tensorboard_writer: _fscore/HAM        |     0.919  |       N/A
2021-04-28 12:22:01,666 - allennlp.training.tensorboard_writer - INFO - _fscore/HAM        |     0.919  |       N/A
INFO:allennlp.training.tensorboard_writer: _fscore/SPAM       |     0.930  |       N/A
2021-04-28 12:22:01,670 - allennlp.training.tensorboard_writer - INFO - _fscore/SPAM       |     0.930  |       N/A
INFO:allennlp.training.tensorboard_writer: _precision/HAM     |     0.928  |       N/A
2021-04-28 12:22:01,677 - allennlp.training.tensorboard_writer - INFO - _precision/HAM     |     0.928  |       N/A
INFO:allennlp.training.tensorboard_writer: _precision/SPAM    |     0.923  |       N/A
2021-04-28 12:22:01,680 - allennlp.training.tensorboard_writer - INFO - _precision/SPAM    |    

...


INFO:allennlp.training.tensorboard_writer:                        Training |  Validation
2021-04-28 12:22:18,212 - allennlp.training.tensorboard_writer - INFO -                        Training |  Validation
INFO:allennlp.training.tensorboard_writer: _fscore/HAM        |     1.000  |       N/A
2021-04-28 12:22:18,215 - allennlp.training.tensorboard_writer - INFO - _fscore/HAM        |     1.000  |       N/A
INFO:allennlp.training.tensorboard_writer: _fscore/SPAM       |     1.000  |       N/A
2021-04-28 12:22:18,218 - allennlp.training.tensorboard_writer - INFO - _fscore/SPAM       |     1.000  |       N/A
INFO:allennlp.training.tensorboard_writer: _precision/HAM     |     1.000  |       N/A
2021-04-28 12:22:18,222 - allennlp.training.tensorboard_writer - INFO - _precision/HAM     |     1.000  |       N/A
INFO:allennlp.training.tensorboard_writer: _precision/SPAM    |     1.000  |       N/A
2021-04-28 12:22:18,224 - allennlp.training.tensorboard_writer - INFO - _precision/SPAM    |    


INFO:allennlp.training.tensorboard_writer:                        Training |  Validation
2021-04-28 12:22:19,068 - allennlp.training.tensorboard_writer - INFO -                        Training |  Validation
INFO:allennlp.training.tensorboard_writer: _fscore/HAM        |     1.000  |       N/A
2021-04-28 12:22:19,071 - allennlp.training.tensorboard_writer - INFO - _fscore/HAM        |     1.000  |       N/A
INFO:allennlp.training.tensorboard_writer: _fscore/SPAM       |     1.000  |       N/A
2021-04-28 12:22:19,074 - allennlp.training.tensorboard_writer - INFO - _fscore/SPAM       |     1.000  |       N/A
INFO:allennlp.training.tensorboard_writer: _precision/HAM     |     1.000  |       N/A
2021-04-28 12:22:19,078 - allennlp.training.tensorboard_writer - INFO - _precision/HAM     |     1.000  |       N/A
INFO:allennlp.training.tensorboard_writer: _precision/SPAM    |     1.000  |       N/A
2021-04-28 12:22:19,081 - allennlp.training.tensorboard_writer - INFO - _precision/SPAM    |    


INFO:allennlp.models.archival: archiving weights and vocabulary to path_to_store_training_run_output/model.tar.gz
2021-04-28 12:22:19,252 - allennlp.models.archival - INFO - archiving weights and vocabulary to path_to_store_training_run_output/model.tar.gz


best_epoch,19
peak_worker_0_memory_MB,1117.39062
training_duration,0:00:19.689126
training_start_epoch,0
training_epochs,19
epoch,19
training_accuracy,1.0
training_micro/precision,1.0
training_micro/recall,1.0
training_micro/fscore,1.0
training_macro/precision,1.0


best_epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
peak_worker_0_memory_MB,▁▅▅▆▆▆▆▆▇▇▇▇▇▇▇█████
training_start_epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_epochs,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
training_accuracy,▁▆▇█████████████████
training_micro/precision,▁▆▇█████████████████
training_micro/recall,▁▆▇█████████████████
training_micro/fscore,▁▆▇█████████████████
training_macro/precision,▁▆▇█████████████████
training_macro/recall,▁▆▇█████████████████


Once the process has finished, we can go to the "path_to_store_training_run_output" folder and open our metrics.json to see all the information. We won't show everything, but make sure to check all the information available.

In [52]:
import json

with open("./path_to_store_training_run_output/metrics.json") as json_file:
    metrics = json.load(json_file)
    
print("Simple pipeline test accuracy:", metrics["test_accuracy"])

Simple pipeline test accuracy: 0.868


In [53]:
pipeline.predict("Don't forget to check out biome.text for more information :)")

{'labels': ['SPAM', 'HAM'],
 'probabilities': [0.9997109770774841, 0.00028897481388412416]}

## Summary

In this tutorial, we accomplished the following:

- We introduced the concept of Labeling Functions (LFs) and demonstrated some of the forms they can take.
- We used the Snorkel LabelModel to automatically learn how to combine the outputs of our LFs into strong probabilistic labels.
- We showed how we can incorporate Rubrix into our workflow to get faster and more accurate results.
- We built and tested a simple pipeline using biome.text

## What's next

TODO: No need to add anything here yet